In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# Auswahloption: Beispiel verwenden oder eigene Datei hochladen
option_selector = widgets.ToggleButtons(
    options=[('Use example instance', 'example'), ('Upload your own', 'upload')],
    description='Choose input:',
    style={'description_width': 'initial'}
)

# Upload-Widget für benutzerdefinierte TTL-Datei
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# Output-Bereich für Rückmeldungen
output = widgets.Output()

# Variable zur Speicherung des ausgewählten Dateipfads
instance_file_path = None

# Funktion zur Auswahlbehandlung
def on_selection_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = os.path.join("Instance_Files", "swemls-instances.ttl")
            print(f"✅ Example file selected: {instance_file_path}")
        elif change['new'] == 'upload':
            print("📁 Please upload your instance TTL file below:")
            display(upload_widget)

# Event Listener für die Auswahloptionen
option_selector.observe(on_selection_change, names='value')

# Funktion zur Behandlung des Dateiuploads
def on_file_upload(change):
    global instance_file_path
    uploaded_file = list(upload_widget.value.values())[0]
    instance_file_path = uploaded_file['metadata']['name']
    with open(instance_file_path, 'wb') as f:
        f.write(uploaded_file['content'])
    output.clear_output()
    with output:
        print(f"✅ File uploaded and saved as: {instance_file_path}")

upload_widget.observe(on_file_upload, names='value')

# Anzeige der UI-Elemente
display(option_selector, output)


In [ ]:
from rdflib import Graph, Namespace
import os

# SHACL & SWeMLS namespaces
SH = Namespace("http://www.w3.org/ns/shacl#")
SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")

# Verzeichnis der SHACL-Regeln (relativ zum Projektordner)
shapes_dir = "Shapes"

# Liste der Shape-Dateien
shape_files = [
    "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
    "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
    "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
    "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
    "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
    "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
    "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
    "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
    "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
    "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
    "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl",
    "Y4-shapes.ttl"
]

# UI-Outputbereich für die SHACL-Ausgabe
shacl_output = widgets.Output()

# Parsen und Regelanwendung ausführen
def apply_shacl_rules(instance_file_path):
    g_instance = Graph()
    try:
        g_instance.parse(instance_file_path, format="turtle")
    except Exception as e:
        with shacl_output:
            print(f"❌ Error parsing instance file: {e}")
        return None
    
    triples_before = len(g_instance)

    with shacl_output:
        print(f"📥 Parsed instance file: {instance_file_path}")
        print(f"📊 Triples before applying SHACL: {triples_before}")
        print(f"🔍 Applying SHACL rules...")

    # SHACL-Regeln anwenden
    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_file}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error parsing shape file {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                    with shacl_output:
                        print(f"✅ Executed rule from {shape_file}")
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Error executing rule from {shape_file}: {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print(f"✅ All rules applied.")
        print(f"📈 Triples after applying SHACL: {triples_after}")
        print(f"➕ {triples_after - triples_before} new triples inferred.")

    return g_instance  # Rückgabe für spätere Verarbeitung (z. B. Query)

# Anwendung nur, wenn Datei vorhanden
if instance_file_path:
    result_graph = apply_shacl_rules(instance_file_path)
    display(shacl_output)
else:
    display(widgets.HTML("<b>⚠️ Please select or upload a TTL file first.</b>"))


In [ ]:
from rdflib.namespace import RDF
import ipywidgets as widgets
from IPython.display import display, clear_output

# Output für alles rund um SPARQL-Interaktion
query_interface_output = widgets.Output()

# Textfeld für SPARQL-Eingabe
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>

SELECT DISTINCT ?system
WHERE {
  ?system a swemls:System .
}
""",
    placeholder='Enter your SPARQL query here...',
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Button zur Ausführung
run_query_button = widgets.Button(
    description='Run Query',
    button_style='primary'
)

# Dropdown & Auswahl
system_selector = widgets.Dropdown(description="Select system:")
confirm_button = widgets.Button(description="Confirm selection", button_style='success')
selection_output = widgets.Output()

# Handler für Query-Button
def on_query_run(b):
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if not result_graph:
        with query_interface_output:
            print("⚠️ RDF graph not loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ Query error: {e}")
        return

    matched_systems = []
    for row in results:
        uri = str(row.system)
        sys_id = uri.split("/")[-1]
        matched_systems.append((sys_id, uri))

    if not matched_systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = [(sys_id, uri) for sys_id, uri in matched_systems]

    with query_interface_output:
        print(f"✅ Found {len(matched_systems)} matching system(s). Please select one:")

    display(system_selector, confirm_button)

# Handler für Bestätigung
def on_confirm_selection(b):
    selected_label = system_selector.label
    selected_uri = system_selector.value
    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_label}")
        global selected_system_uri
        selected_system_uri = selected_uri
        global selected_system_id
        selected_system_id = selected_label

run_query_button.on_click(on_query_run)
confirm_button.on_click(on_confirm_selection)

# Anzeige der gesamten Benutzeroberfläche
display(query_input, run_query_button, query_interface_output, selection_output)
